# Chess Position Scorer (GPU-Accelerated with Lc0)

This notebook scores chess positions using Leela Chess Zero (Lc0) with GPU acceleration.

**Setup time:** ~10 minutes (one-time build)

**Scoring speed:** ~1-2 positions/second on A100

---

## IMPORTANT: Enable GPU First!

Go to **Runtime → Change runtime type → GPU (A100 if available)**

In [ ]:
# Step 1: Check GPU is available
!nvidia-smi --query-gpu=name,memory.total --format=csv
print("\nIf you see 'NVIDIA-SMI has failed', go to Runtime > Change runtime type > GPU")

In [ ]:
# Step 2: Install build dependencies
!apt-get update -qq
!apt-get install -qq -y git ninja-build clang libopenblas-dev
!pip install -q meson chess

print("\nDependencies installed!")

In [ ]:
# Step 3: Build Lc0 from source (~5-10 minutes)
import os

if not os.path.exists("lc0"):
    print("Cloning Lc0 repository...")
    !git clone --depth 1 https://github.com/LeelaChessZero/lc0.git
else:
    print("Lc0 repository already exists, skipping clone...")

print("\nBuilding Lc0 with CUDA backend...")
print("(This takes ~5-10 minutes, please wait)\n")

%cd lc0
!CC=clang CXX=clang++ ./build.sh
%cd ..

LC0_PATH = "/content/lc0/build/release/lc0"

if os.path.exists(LC0_PATH):
    print(f"\n✓ Lc0 built successfully!")
    print(f"  Binary: {LC0_PATH}")
else:
    print("\n✗ Build failed! Check errors above.")

In [ ]:
# Step 4: Download neural network weights
import urllib.request
import gzip
import shutil
import os

NETWORK_FILE = "/content/network.pb"

if not os.path.exists(NETWORK_FILE):
    NETWORK_URL = "https://storage.lczero.org/files/networks-contrib/t1-512x15x8h-distilled-swa-3395000.pb.gz"
    
    print("Downloading neural network weights (~40MB)...")
    urllib.request.urlretrieve(NETWORK_URL, "network.pb.gz")
    
    print("Extracting...")
    with gzip.open("network.pb.gz", "rb") as f_in:
        with open(NETWORK_FILE, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
    os.remove("network.pb.gz")
    print(f"\n✓ Network downloaded: {os.path.getsize(NETWORK_FILE) / 1024 / 1024:.1f} MB")
else:
    print(f"✓ Network already exists: {NETWORK_FILE}")

NETWORK_PATH = NETWORK_FILE
LC0_PATH = "/content/lc0/build/release/lc0"

In [ ]:
# Step 5: Test Lc0
LC0_PATH = "/content/lc0/build/release/lc0"
NETWORK_PATH = "/content/network.pb"

print("Testing Lc0 engine...")
!{LC0_PATH} --weights={NETWORK_PATH} --backend=cuda-auto -t 1 -v 2>&1 | head -20

print("\n✓ Lc0 is ready!")

---
## Mount Google Drive & Set Positions Path

In [ ]:
# Step 6: Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

print("\nGoogle Drive mounted at /content/drive/MyDrive/")

In [ ]:
# Step 7: Set your positions directory
# ================================================
# CHANGE THIS PATH to match your Google Drive folder
# ================================================

POSITIONS_DIR = "/content/drive/MyDrive/chess_positions"

# ================================================

import os
import glob

if not os.path.exists(POSITIONS_DIR):
    print(f"Creating directory: {POSITIONS_DIR}")
    os.makedirs(POSITIONS_DIR, exist_ok=True)
    print("\n⚠ Directory was empty! Upload your position_*.txt files to this folder.")
else:
    position_files = sorted(glob.glob(f"{POSITIONS_DIR}/position_*.txt"))
    position_files = [f for f in position_files if "_score" not in f]
    score_files = sorted(glob.glob(f"{POSITIONS_DIR}/position_*_score.txt"))
    
    print(f"Directory: {POSITIONS_DIR}")
    print(f"Position files found: {len(position_files)}")
    print(f"Already scored: {len(score_files)}")
    print(f"Remaining to score: {len(position_files) - len(score_files)}")

---
## Scoring Engine & Configuration

In [ ]:
# Scoring engine class
import subprocess
import math
import time
import sys

class Lc0Engine:
    """Wrapper for Lc0 engine with UCI protocol."""
    
    def __init__(self, lc0_path, network_path, nodes=800, backend="cuda-auto"):
        self.nodes = nodes
        self.process = subprocess.Popen(
            [lc0_path, f"--weights={network_path}", f"--backend={backend}", "--verbose-move-stats"],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1
        )
        self._send("uci")
        self._wait_for("uciok")
        self._send("isready")
        self._wait_for("readyok")
    
    def _send(self, cmd):
        self.process.stdin.write(cmd + "\n")
        self.process.stdin.flush()
    
    def _wait_for(self, expected):
        while True:
            line = self.process.stdout.readline().strip()
            if expected in line:
                return line
    
    def _read_until_bestmove(self):
        info_lines = []
        while True:
            line = self.process.stdout.readline().strip()
            if line.startswith("info"):
                info_lines.append(line)
            elif line.startswith("bestmove"):
                return info_lines, line
    
    def analyze(self, fen):
        self._send(f"position fen {fen}")
        self._send(f"go nodes {self.nodes}")
        
        info_lines, bestmove_line = self._read_until_bestmove()
        
        best_move = bestmove_line.split()[1] if len(bestmove_line.split()) > 1 else None
        
        wdl = None
        cp = None
        
        for line in reversed(info_lines):
            if "wdl" in line and wdl is None:
                parts = line.split()
                try:
                    wdl_idx = parts.index("wdl")
                    w = int(parts[wdl_idx + 1])
                    d = int(parts[wdl_idx + 2])
                    l = int(parts[wdl_idx + 3])
                    wdl = (w / 1000, d / 1000, l / 1000)
                except (ValueError, IndexError):
                    pass
            
            if "score cp" in line and cp is None:
                parts = line.split()
                try:
                    cp_idx = parts.index("cp")
                    cp = int(parts[cp_idx + 1])
                except (ValueError, IndexError):
                    pass
            
            if wdl is not None:
                break
        
        if cp is None and wdl:
            cp = self.wdl_to_cp(wdl)
        
        return {"wdl": wdl, "cp": cp, "best_move": best_move, "nodes": self.nodes}
    
    @staticmethod
    def wdl_to_cp(wdl):
        if wdl is None:
            return None
        w, d, l = wdl
        win_rate = w + d / 2
        win_rate = max(0.001, min(0.999, win_rate))
        cp = 111.714 * math.tan(1.5620688 * (win_rate - 0.5))
        return int(round(cp))
    
    def close(self):
        try:
            self._send("quit")
            self.process.wait(timeout=5)
        except:
            self.process.kill()


def format_cp(cp):
    if cp is None:
        return "?"
    pawns = cp / 100
    return f"{pawns:+.2f}"


def format_time(seconds):
    if seconds < 60:
        return f"{seconds:.0f}s"
    elif seconds < 3600:
        return f"{int(seconds // 60)}m {int(seconds % 60)}s"
    else:
        return f"{int(seconds // 3600)}h {int((seconds % 3600) // 60)}m"


print("✓ Scoring engine ready!")

In [ ]:
# Configuration
# ================================================

NODES = 800        # Nodes per position:
                   #   400  = fast (~0.3s)
                   #   800  = balanced (~0.7s)
                   #   1600 = strong (~1.5s)
                   #   5000 = very strong (~4s)

FORCE_RESCORE = False  # Set True to re-score already scored positions

# ================================================

print(f"Configuration:")
print(f"  Nodes per position: {NODES}")
print(f"  Force re-score: {FORCE_RESCORE}")
print(f"  Positions directory: {POSITIONS_DIR}")

---
## Run Scoring

In [ ]:
# Find positions to score
import os
import glob

position_files = sorted(glob.glob(f"{POSITIONS_DIR}/position_*.txt"))
position_files = [f for f in position_files if "_score" not in f]

if not FORCE_RESCORE:
    to_score = []
    for pf in position_files:
        score_file = pf.replace(".txt", "_score.txt")
        if not os.path.exists(score_file):
            to_score.append(pf)
    already_scored = len(position_files) - len(to_score)
else:
    to_score = position_files
    already_scored = 0

print(f"Total positions: {len(position_files)}")
print(f"Already scored: {already_scored}")
print(f"To score: {len(to_score)}")

if not to_score:
    print("\n✓ All positions already scored!")
    print("  Set FORCE_RESCORE = True to re-score.")

In [ ]:
# Run scoring
LC0_PATH = "/content/lc0/build/release/lc0"
NETWORK_PATH = "/content/network.pb"

if to_score:
    print(f"Initializing Lc0 engine (nodes={NODES})...")
    engine = Lc0Engine(LC0_PATH, NETWORK_PATH, nodes=NODES)
    print("Engine ready!\n")
    
    scored = 0
    errors = 0
    start_time = time.time()
    
    try:
        for i, pos_file in enumerate(to_score, 1):
            try:
                with open(pos_file, "r") as f:
                    fen = f.read().strip()
                
                result = engine.analyze(fen)
                
                score_file = pos_file.replace(".txt", "_score.txt")
                cp_str = format_cp(result["cp"])
                
                wdl_str = ""
                if result['wdl']:
                    wdl_str = f"{result['wdl'][0]:.1%}/{result['wdl'][1]:.1%}/{result['wdl'][2]:.1%}"
                
                score_content = f"{cp_str}\n{result['best_move']}\n{NODES}\n{wdl_str}"
                
                with open(score_file, "w") as f:
                    f.write(score_content)
                
                scored += 1
                
                elapsed = time.time() - start_time
                rate = scored / elapsed if elapsed > 0 else 0
                eta = (len(to_score) - i) / rate if rate > 0 else 0
                progress = i / len(to_score) * 100
                
                print(f"\r[{progress:5.1f}%] {i}/{len(to_score)} | {format_time(elapsed)} | ETA: {format_time(eta)} | {rate:.1f}/s | {cp_str}", end="", flush=True)
                
            except Exception as e:
                errors += 1
                print(f"\nError on {os.path.basename(pos_file)}: {e}")
    
    except KeyboardInterrupt:
        print("\n\n⚠ Interrupted by user")
    
    finally:
        engine.close()
    
    elapsed = time.time() - start_time
    print(f"\n\n{'='*50}")
    print(f"✓ Scoring complete!")
    print(f"  Scored: {scored}")
    print(f"  Errors: {errors}")
    print(f"  Time: {format_time(elapsed)}")
    print(f"  Speed: {scored/elapsed:.2f} positions/second")
    print(f"  Output: {POSITIONS_DIR}")
else:
    print("Nothing to score.")

---
## View Results

In [ ]:
# Check results
import glob

score_files = sorted(glob.glob(f"{POSITIONS_DIR}/position_*_score.txt"))
print(f"Total score files: {len(score_files)}")

if score_files:
    print(f"\n--- Sample: {os.path.basename(score_files[0])} ---")
    with open(score_files[0], "r") as f:
        content = f.read()
        print(content)
    
    print("\n--- Format ---")
    print("Line 1: Centipawn score (+0.45 = white +0.45 pawns)")
    print("Line 2: Best move (e2e4)")
    print("Line 3: Nodes searched")
    print("Line 4: Win/Draw/Loss percentages")

In [ ]:
# Optional: Download as ZIP
# Uncomment to download all scored positions

# import shutil
# from google.colab import files
# 
# shutil.make_archive("/content/scored_positions", "zip", POSITIONS_DIR)
# files.download("/content/scored_positions.zip")
# print("Download started!")